Using code from the following github repo: 
- https://github.com/codegiovanni/Analog_clock/blob/main/analog_clock.py
- https://github.com/theRealProHacker/PyGameRecorder
- http://github.com/ArjunSahlot/vidmaker

In [2]:
# Import packages
import pygame
from math import pi, cos, sin, atan2, radians
import datetime

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [120]:
# Functions
def draw_value(value, font, size, position, color, bold=True, italic=False):
    font = pygame.font.SysFont(font, size, bold, italic)
    text = font.render(value, True, color)
    text_rect = text.get_rect(center=(position))
    screen.blit(text, text_rect)

def polar_to_cartesian(r, theta, xy_0=(0, 0)):
    x = r * sin(pi * theta / 180) + xy_0[0]
    y = -r * cos(pi * theta / 180) + xy_0[1]
    return x, y

def draw_arrow(screen, color, start, r, theta, line_width, arrow_size=50, arrow_angle=25):
    """Draw an arrow on the given surface from start to end."""
    line_end = polar_to_cartesian(r-arrow_size/2, theta, xy_0=start)
    arrow_end = polar_to_cartesian(r, theta, xy_0=start)
    # Draw line
    pygame.draw.line(screen, color, start, line_end, line_width)

    # Calculate the angle of the line
    line_angle = atan2(arrow_end[1] - start[1], arrow_end[0] - start[0])

    # Calculate the points for the arrowhead
    arrow_p1 = (arrow_end[0] - arrow_size * cos(line_angle - radians(arrow_angle)),
                arrow_end[1] - arrow_size * sin(line_angle - radians(arrow_angle)))
    arrow_p2 = (arrow_end[0] - arrow_size * cos(line_angle + radians(arrow_angle)),
                arrow_end[1] - arrow_size * sin(line_angle + radians(arrow_angle)))

    # Draw the arrowhead as a filled polygon
    pygame.draw.polygon(screen, color, (arrow_end, arrow_p1, arrow_p2))

def draw_clock_base(screen, colors, center, clock_radius):
    screen.fill(colors['white'])
    pygame.draw.circle(screen, colors['black'], center, clock_radius-5, 10)
    pygame.draw.circle(screen, colors['black'], center, 12)
    # pygame.draw.rect(screen, colors['black'], [WIDTH / 2 - 260, HEIGHT / 2 - 30, 80, 60], 1)

    for number in range(1, 13):
        draw_value(str(number), "Arial", 80, polar_to_cartesian(clock_radius-80, number*30, xy_0=center), colors['black'])

    for number in range(0, 360, 6):
        if number % 5:
            pygame.draw.line(screen, colors['black'], polar_to_cartesian(clock_radius - 15, number, xy_0=center),
                             polar_to_cartesian(clock_radius - 30, number, xy_0=center), 2)
        else:
            pygame.draw.line(screen, colors['black'], polar_to_cartesian(clock_radius - 15, number, xy_0=center),
                             polar_to_cartesian(clock_radius - 35, number, xy_0=center), 6)

def draw_re_im_axes(screen, colors, center, clock_radius, axes_radius_buf, axes_linewidth):
    # Draw Real and Imaginary Axes
    draw_arrow(screen, colors['red'], center, clock_radius+axes_radius_buf, 0, axes_linewidth, arrow_size=axes_linewidth*6)
    draw_arrow(screen, colors['red'], center, clock_radius+axes_radius_buf, 180, axes_linewidth, arrow_size=axes_linewidth*6)     
    draw_arrow(screen, colors['blue'], center, clock_radius+axes_radius_buf, 90, axes_linewidth, arrow_size=axes_linewidth*6)
    draw_arrow(screen, colors['blue'], center, clock_radius+axes_radius_buf, 270, axes_linewidth, arrow_size=axes_linewidth*6)  
    # Draw Real (Re) and Imaginary (Im) labels
    draw_value('+Im', "latinmodernmath", 30, polar_to_cartesian(clock_radius+axes_radius_buf+5, 4, xy_0=center), colors['red'], bold=False)
    draw_value('-Im', "latinmodernmath", 30, polar_to_cartesian(clock_radius+axes_radius_buf+5, 184, xy_0=center), colors['red'], bold=False)
    draw_value('+Re', "latinmodernmath", 30, polar_to_cartesian(clock_radius+axes_radius_buf+10, 87, xy_0=center), colors['blue'], bold=False)
    draw_value('-Re', "latinmodernmath", 30, polar_to_cartesian(clock_radius+axes_radius_buf+10, 267, xy_0=center), colors['blue'], bold=False)
    # Draw tick marks showing scale
    # Imaginary 
    im_scale_loc0 = polar_to_cartesian(clock_radius, 0, xy_0=center)
    im_scale_loc1 = (im_scale_loc0[0]-12, im_scale_loc0[1])
    pygame.draw.line(screen, colors['red'], im_scale_loc0, im_scale_loc1, 4)
    draw_value('1', "latinmodernmath", 30, polar_to_cartesian(clock_radius+7,356, xy_0=center), colors['red'])
    # Real
    re_scale_loc0 = polar_to_cartesian(clock_radius, 90, xy_0=center)
    re_scale_loc1 = (re_scale_loc0[0], re_scale_loc0[1]+12)
    pygame.draw.line(screen, colors['blue'], re_scale_loc0, re_scale_loc1, 4)
    draw_value('1', "latinmodernmath", 30, polar_to_cartesian(clock_radius+7, 94, xy_0=center), colors['blue'])
    
def print_available_math_fonts():
    print(*[fnt for fnt in pygame.font.get_fonts() if 'math' in fnt], sep='\n')

In [164]:
# Set parameters

start_time = "now"
# start_time = "12:00:00"
# time_format = "%H:%M:%S"
# time_delta = {'hours': 0, 'minutes': 0, 'seconds': 1}
fps = 1

screen_w_h = (960, 960)
clock_radius = 400
r_ratio_h_m_s = (.65, .75, .9)
linewidth_h_m_s = (16, 12, 6)

colors = {'white': (255, 255, 255), \
          'black': (0, 0, 0), \
          'red': (255, 0, 0), \
          'blue': (0, 0, 255)}

axes_radius_buf = 30
axes_linewidth = 4

In [165]:
# Run main function
center = (screen_w_h[0] / 2, screen_w_h[1] / 2)
pygame.init()

screen = pygame.display.set_mode(screen_w_h)
pygame.display.set_caption("Complex Number Clock")
clock = pygame.time.Clock()

def main():
    if start_time != "now":
        clock_time = datetime.datetime.strptime(start_time, time_format)
    run = True
    while run:

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        
        draw_clock_base(screen, colors, center, clock_radius)
        
        draw_re_im_axes(screen, colors, center, clock_radius, axes_radius_buf, axes_linewidth)
        
        if start_time == "now":
            clock_time = datetime.datetime.now()
        else:
            try:
                clock_time += datetime.timedelta(**time_delta)
            except Exception as err:
                print(err)
                raise
        second = clock_time.second
        minute = clock_time.minute
        hour = clock_time.hour
        
        lw_h, lw_m, lw_s = linewidth_h_m_s
        # Hour
        h_theta = (hour + minute / 60 + second / 3600) * (360 / 12)
        draw_arrow(screen, colors['black'], center, clock_radius*r_ratio_h_m_s[0], h_theta, lw_h, arrow_size=lw_h*4)

        # Minute
        m_theta = (minute + second / 60) * (360 / 60)
        draw_arrow(screen, colors['black'], center, clock_radius*r_ratio_h_m_s[1], m_theta, lw_m, arrow_size=lw_m*4)
        
        # Second
        s_theta = second * (360 / 60)
        pygame.draw.line(screen, colors['black'], center, polar_to_cartesian(clock_radius*r_ratio_h_m_s[2], s_theta, xy_0=center), lw_s)
        
        pygame.display.update()

        clock.tick(fps)

    pygame.quit()

main()

In [160]:
# Quit if there is an error to close screen
pygame.quit()